In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from whispervq_conv import WhisperForConditionalGeneration
from transformers import AutoFeatureExtractor, AutoTokenizer, AutoProcessor
import librosa
import torch

torch.set_grad_enabled(False)

In [3]:
WhisperForConditionalGeneration.register_for_auto_class("AutoModel")

In [4]:
!ls whisper-conv-large-v3-turbo-vq

checkpoint-279000  checkpoint-279500  checkpoint-280000  runs


In [5]:
model = WhisperForConditionalGeneration.from_pretrained(
    'whisper-conv-large-v3-turbo-vq/checkpoint-280000', torch_dtype = 'auto')

In [6]:
_ = model.cuda()

In [7]:
feature_extractor = AutoFeatureExtractor.from_pretrained('openai/whisper-large-v3-turbo')
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3-turbo')
processor = AutoProcessor.from_pretrained('openai/whisper-large-v3-turbo')

In [8]:
from streaming import MDSWriter, LocalDataset

dataset = LocalDataset('mosaic-stt-include-malaysian')

In [9]:
dataset[-100003]['audio_filename'], dataset[-100002]['audio_filename']

('audio/fr/dev/audio/common_voice_fr_19722301.mp3',
 'audio/fr/dev/audio/common_voice_fr_19722302.mp3')

In [10]:
encoder = model.model.get_encoder()

In [11]:
y, sr = librosa.load('common_voice_ba_26517811.mp3', sr = feature_extractor.sampling_rate)
features = feature_extractor([y], return_tensors = 'pt', return_attention_mask = True)
for k in features.keys():
    features[k] = features[k].cuda()
encoded = encoder(**features)
encoded[1][0, encoded[2][0] == 1]

It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


tensor([14135,  7585, 12890, 32383, 15559,  4515,   252, 32713,   252, 16296,
         3050, 18175, 15733,  5619,  5619,  1770,  7520, 32041, 26287,  8139,
         8453, 28652,  4327, 26837, 20927, 26620, 12310, 12310, 12938, 29755,
        29755, 18102,  5597,  8076,  8076,  8076,  9772, 31738, 31738,  1856,
        24397, 27124,  5538,  1970, 29984,  8891, 20453, 20453,  1815,  1465,
         1465, 26893,  5597,  9531, 11871, 11871,  6484, 21016, 14653, 18417,
         9598,  9598, 30138, 27531, 18071, 18071, 30147, 24892,   434, 16557,
        30589, 25516, 30876, 30876, 32039, 29394, 27996, 10042,  1939, 16692,
         8163, 16665, 16665,  4507, 28100, 31251,  3051,  3051, 12157, 19865,
        27147, 27357, 21524, 19750, 20016,  9031, 20016, 13475, 30149, 30149,
        21785,  4176, 24032, 19334, 17387, 31375,  2659, 16509, 31672,  7785,
        10352, 30063,  8518, 30730, 29357, 28538,  7072], device='cuda:0')

In [12]:
i = 100
t = dataset[i]['text']
y, sr = librosa.load(dataset[i]['audio_filename'], sr = feature_extractor.sampling_rate)
tokenized = tokenizer(dataset[i]['text'], add_special_tokens = False)['input_ids']
input_ids = []
for k in tokenized:
    input_ids.append(k)
    if k == 50364:
        break
features = feature_extractor([y], return_tensors = 'pt', return_attention_mask = True)
features['decoder_input_ids'] = torch.tensor(input_ids)[None]
for k in features.keys():
    features[k] = features[k].cuda()

generate_kwargs = dict(
    **features,
    max_new_tokens=1024,
    temperature=0.0,
    do_sample=False
)
generation_output = model.generate(**generate_kwargs)
tokenizer.decode(generation_output[0]), t

It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


('<|startoftranscript|><|en|><|transcribe|><|notimestamps|> Hey, I got some questions about my vaccination schedule lah. Last time jab first dose Moderna, second dose Pfizer. Now how to continue to book?<|endoftext|>',
 '<|startoftranscript|><|en|><|transcribe|><|notimestamps|> Ehhh, I got some question about my vaccination schedule la. Last time jab first dose Moderna, second dose Pfizer. Now how to continue with booster?<|endoftext|>')

In [13]:
y, sr = librosa.load('common_voice_ba_26517811.mp3', sr = feature_extractor.sampling_rate)
input_ids = tokenizer(
    '<|startoftranscript|><|ru|><|transcribe|><|notimestamps|>', 
    add_special_tokens = False, return_tensors = 'pt')['input_ids']
features = feature_extractor([y], return_tensors = 'pt', return_attention_mask = True)
features['decoder_input_ids'] = input_ids
for k in features.keys():
    features[k] = features[k].cuda()

generate_kwargs = dict(
    **features,
    max_new_tokens=1024,
)
generation_output = model.generate(**generate_kwargs)
tokenizer.decode(generation_output[0])

It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


'<|startoftranscript|><|ru|><|transcribe|><|notimestamps|> Кубах сирта был холква кешене битарафлыг сирпаса.<|endoftext|>'

In [ ]:
y, sr = librosa.load('218757.mp3', sr = feature_extractor.sampling_rate)
input_ids = tokenizer(
    '<|startoftranscript|><|ms|><|transcribe|><|notimestamps|>', 
    add_special_tokens = False, return_tensors = 'pt')['input_ids']
features = feature_extractor([y], return_tensors = 'pt', return_attention_mask = True)
features['decoder_input_ids'] = input_ids
for k in features.keys():
    features[k] = features[k].cuda()

generate_kwargs = dict(
    **features,
    max_new_tokens=1024,
    temperature=0.1,
    do_sample=True
)
generation_output = model.generate(**generate_kwargs)
tokenizer.decode(generation_output[0])

In [6]:
model.push_to_hub('mesolitica/whisper-conv-VQ-32k-large-v3-turbo')

README.md: 0.00B [00:00, ?B/s]

[2025-08-05 08:39:08,343] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
collect2: error: ld returned 1 exit status


Uploading...:   0%|          | 0.00/3.40G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/whisper-conv-VQ-32k-large-v3-turbo/commit/23bea549fbe1f3533d7a05ae4f056b0e2f7441af', commit_message='Upload WhisperForConditionalGeneration', commit_description='', oid='23bea549fbe1f3533d7a05ae4f056b0e2f7441af', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mesolitica/whisper-conv-VQ-32k-large-v3-turbo', endpoint='https://huggingface.co', repo_type='model', repo_id='mesolitica/whisper-conv-VQ-32k-large-v3-turbo'), pr_revision=None, pr_num=None)

In [9]:
tokenizer.push_to_hub('mesolitica/whisper-conv-VQ-32k-large-v3-turbo')

CommitInfo(commit_url='https://huggingface.co/mesolitica/whisper-conv-VQ-32k-large-v3-turbo/commit/d10350d1730ad542954800825dc817ddcfa3a05d', commit_message='Upload tokenizer', commit_description='', oid='d10350d1730ad542954800825dc817ddcfa3a05d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mesolitica/whisper-conv-VQ-32k-large-v3-turbo', endpoint='https://huggingface.co', repo_type='model', repo_id='mesolitica/whisper-conv-VQ-32k-large-v3-turbo'), pr_revision=None, pr_num=None)

In [10]:
feature_extractor.push_to_hub('mesolitica/whisper-conv-VQ-32k-large-v3-turbo')

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/mesolitica/whisper-conv-VQ-32k-large-v3-turbo/commit/d10350d1730ad542954800825dc817ddcfa3a05d', commit_message='Upload feature extractor', commit_description='', oid='d10350d1730ad542954800825dc817ddcfa3a05d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mesolitica/whisper-conv-VQ-32k-large-v3-turbo', endpoint='https://huggingface.co', repo_type='model', repo_id='mesolitica/whisper-conv-VQ-32k-large-v3-turbo'), pr_revision=None, pr_num=None)

In [11]:
processor.push_to_hub('mesolitica/whisper-conv-VQ-32k-large-v3-turbo')

CommitInfo(commit_url='https://huggingface.co/mesolitica/whisper-conv-VQ-32k-large-v3-turbo/commit/901d4247bbb4eaa57fa7a202fee9ed7a8ea45f63', commit_message='Upload processor', commit_description='', oid='901d4247bbb4eaa57fa7a202fee9ed7a8ea45f63', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mesolitica/whisper-conv-VQ-32k-large-v3-turbo', endpoint='https://huggingface.co', repo_type='model', repo_id='mesolitica/whisper-conv-VQ-32k-large-v3-turbo'), pr_revision=None, pr_num=None)